In [1]:
from web3 import Web3
import json
import requests
import csv
import pandas as pd
import seaborn as sns 
import numpy as np
import matplotlib.pyplot as plt
from time import *
import plotly.express as px
import plotly.io as pio
pd.options.plotting.backend = "plotly"
pio.renderers.default = "plotly_mimetype+notebook_connected"
import plotly.graph_objects as go
import math
import warnings
import scipy.stats
import random

## avg_blocksize_stable

In [73]:
df1 = pd.read_csv('bxs_12935_12965.csv')
df2 = pd.read_csv('bxs_12965_13080.csv')
df = pd.concat([df1, df2], axis=0)
df['avg_block_size'] = df['gas_used']/((30)*(10**6))
df

,block_number,gas_limit,gas_used,timestamp,base_fee_per_gas,avg_block_size
0,12935400,14999943,14997353,1627763245,NaN,0.499912
1,12935401,14985296,14974043,1627763247,NaN,0.499135
2,12935402,14970663,14777799,1627763253,NaN,0.492593
3,12935403,14977692,14972181,1627763261,NaN,0.499073
4,12935404,14992317,14980982,1627763301,NaN,0.499366
...,...,...,...,...,...,...
114996,13079898,30092779,6697819,1629699470,5.076536e+10,0.223261
114997,13079997,30176087,5077043,1629700690,4.433819e+10,0.169235
114998,13079998,30146620,24003193,1629700697,4.066086e+10,0.800106
114999,13079899,30122165,24210715,1629699476,4.724443e+10,0.807024


In [74]:
stable_blocks = pd.read_csv('blocksize_ref.csv').iloc[:,0]
stable_blocks = list(set(stable_blocks))
stable_blocks = list(map(int,stable_blocks))

In [75]:
df_final = df[df['block_number'].isin(stable_blocks)]
df_final = df_final[["block_number","avg_block_size"]]
df_final.sort_values(by="block_number", ascending=True, inplace=True)
df_final.to_csv('avg_blocksize_stable.csv')

## avg_price

In [37]:
df1 = pd.read_csv('txs_12935_12965.csv')
df2 = pd.read_csv('txs_12965_13080.csv')
df = pd.concat([df1, df2], axis=0)

all_blocks = pd.read_csv('1.csv').iloc[:,0]
all_blocks = list(set(all_blocks))
all_blocks = list(map(int,all_blocks))

burst_blocks = pd.read_csv('11.csv').iloc[:,0]
burst_blocks = list(set(burst_blocks))
burst_blocks = list(map(int,burst_blocks))

stable_blocks = pd.read_csv('12.csv').iloc[:,0]
stable_blocks = list(set(stable_blocks))
stable_blocks = list(map(int,stable_blocks))

df_all = df[df['block_number'].isin(all_blocks)]

df_all.to_csv('txs.csv')

In [38]:
df_bursty = df[df['block_number'].isin(burst_blocks)]
df_stable = df[df['block_number'].isin(stable_blocks)]
df_bursty.to_csv('bursty.csv')
df_stable.to_csv('stable.csv')

## avg_base_fee

In [51]:
df = pd.read_csv('bxs_12965_13080.csv')
# df['basefee'] = df['gas_used']*df['base_fee_per_gas']
df['basefee'] = df['base_fee_per_gas']*0.000000001
df = df[['block_number', 'basefee']]

df_bursty = df[df['block_number'].isin(burst_blocks)]
df_stable = df[df['block_number'].isin(stable_blocks)]

df_bursty.sort_values(by="block_number", ascending=True, inplace=True)
df_stable.sort_values(by="block_number", ascending=True, inplace=True)

df_bursty.to_csv('avg_basefee_burst.csv')
df_stable.to_csv('avg_basefee_stable.csv')

<ipython-input-51-02e1d07748f9>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-51-02e1d07748f9>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## smoothing

In [66]:
def rollingMedian(x,start,end):
    values = []
    for i in range(start,end+1):
        if(i>=0 and i<len(x)):
            values.append(x[i][1])
    values.sort()
    n = len(values)
    if (n == 0):
        return 0
    elif (n%2 == 1):
        return values[int((n-0.5)/2)]
    else:
        return (values[int(n/2)] + values[int(n/2+1)])*0.5
    
def medianSmoother(x,halfwidth):
    n = len(x)
    y = []
    
    for i in range(n):
        lst = [0,0]
        y.append(lst)
        
    for i in range(n):
        y[i][0] = x[i][0];
        y[i][1] = rollingMedian(x, i-halfwidth,i+halfwidth);
        
    return y

In [79]:
halfwidth = 10
x = pd.read_csv('avg_price_all.csv')
x = x.to_numpy()

y = medianSmoother(x,halfwidth)
df = pd.DataFrame.from_records(y, columns = ['block','value'])
df.to_csv('avg_price_all_smoothed.csv')

x = pd.read_csv('avg_blocksize_stable.csv')
x = x.to_numpy()

y = medianSmoother(x,halfwidth)
df = pd.DataFrame.from_records(y, columns = ['block','value'])
df.to_csv('avg_blocksize_stable_smoothed.csv')